In [1]:
# import libraries
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# import encoded dataset
data = pd.read_csv(r"/Users/kseniiailchenko/Downloads/Project7_Netflix-main/data_clean_encoded.csv")

In [3]:
data.head()

,Unnamed: 0,title,year,kind,rating,vote,runtime,genre1,country1,language1,cast1,cast2,director1,writer1
0,0,1765,2003.0,2,7.7,474.0,50.0,6,62,16,867,4968,4130,3319
1,1,2416,1994.0,7,8.1,18.0,83.0,8,62,16,3627,438,3672,5041
2,2,75,1992.0,1,5.5,93.0,95.0,0,37,33,2513,1667,4077,2610
3,3,7538,2004.0,1,5.3,13432.0,60.0,6,62,16,3047,1545,4022,4892
4,4,1341,1991.0,1,4.5,2177.0,96.0,4,62,16,639,3350,1124,2893


In [4]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
data.rating = data.rating.astype(int)
data

,title,year,kind,rating,vote,runtime,genre1,country1,language1,cast1,cast2,director1,writer1
0,1765,2003.0,2,7,474.0,50.0,6,62,16,867,4968,4130,3319
1,2416,1994.0,7,8,18.0,83.0,8,62,16,3627,438,3672,5041
2,75,1992.0,1,5,93.0,95.0,0,37,33,2513,1667,4077,2610
3,7538,2004.0,1,5,13432.0,60.0,6,62,16,3047,1545,4022,4892
4,1341,1991.0,1,4,2177.0,96.0,4,62,16,639,3350,1124,2893
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8130,354,1969.0,1,3,4384.0,95.0,1,37,33,4643,2127,1565,34
8131,2168,2005.0,0,6,44.0,93.0,6,67,16,3822,1799,56,71
8132,2035,2001.0,3,4,2376.0,94.0,19,62,16,1133,5149,2570,2426
8133,5969,2003.0,1,6,6501.0,95.0,7,62,16,3420,3489,3278,3461


In [6]:
# split dataset into independent and dependent variables
X = data.drop('rating', axis=1)
y = data['rating']

In [7]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
STOP

NameError: name 'STOP' is not defined

### Models to import

In [40]:
# import models
from sklearn import ensemble, datasets, tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn import metrics

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

### Feature Selection

In [54]:
# Find the optimal number of features with SFM
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

SFM = SelectFromModel(estimator=RandomForestClassifier())
s = SFM.fit(X, y)

n_features = s.transform(X).shape[1]
n_features

3

In [55]:
# Get the 3 feature names 
feature_idx = s.get_support()
feature_name = X.columns[feature_idx]
feature_name

Index(['title', 'vote', 'cast1'], dtype='object')

In [56]:
# split dataset with features
X = data[['title', 'vote', 'cast1']]
y = data['rating']

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### BaggingClassifier

#### with default parameters

In [57]:
# fitting BaggingClassifier to the dataset
from sklearn.ensemble import BaggingClassifier

bc = BaggingClassifier()
bc.fit(X_train, y_train)

BaggingClassifier()

In [58]:
# model is overfitting training dataset
print('Training Accuracy : %.3f'%bc.score(X_train, y_train))
print('Test Accuracy : %.3f'%bc.score(X_test, y_test))

Training Accuracy : 0.982
Test Accuracy : 0.347


#### with hyperparameter

In [59]:
# GridsearchCV with DecisionTree (default base estimator)
params = {'base_estimator': [None],
         'n_estimators': np.arange(100,400,10)
         }

bc_grid = GridSearchCV(BaggingClassifier(random_state=1), 
                       param_grid = params, 
                       scoring="accuracy", 
                       cv=3, n_jobs=-1, verbose=1)
bc_grid.fit(X_train, y_train)

print(bc_grid.best_params_)
print(bc_grid.best_score_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
{'base_estimator': None, 'n_estimators': 240}
0.35387207962499084


In [60]:
# Accuracy score on test data is still low
bc = BaggingClassifier(n_estimators=240)
bc.fit(X_train, y_train)

print('Training Accuracy : %.3f'%bc.score(X_train, y_train))
print('Test Accuracy : %.3f'%bc.score(X_test, y_test))

Training Accuracy : 1.000
Test Accuracy : 0.370


#### with hyperparameter on a different base_estimator

In [63]:
n_samples = data.shape[0]
n_features = data.shape[1]

params = {'base_estimator': [None],
          'n_estimators': [20,50,100],
          'max_samples': [0.5, 1.0, n_samples//2, ],
          'bootstrap': [True, False],
          'bootstrap_features': [True, False]}

bagging_classifier_grid = GridSearchCV(BaggingClassifier(random_state=1, n_jobs=-1), param_grid =params, cv=3, n_jobs=-1, verbose=1)
bagging_classifier_grid.fit(X_train, y_train)

print('Train Accuracy : %.3f'%bagging_classifier_grid.best_estimator_.score(X_train, y_train))
print('Test Accuracy : %.3f'%bagging_classifier_grid.best_estimator_.score(X_test, y_test))
print('Best Accuracy Through Grid Search : %.3f'%bagging_classifier_grid.best_score_)
print('Best Parameters : ',bagging_classifier_grid.best_params_)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
Train Accuracy : 1.000
Test Accuracy : 0.377
Best Accuracy Through Grid Search : 0.377
Best Parameters :  {'base_estimator': None, 'bootstrap': False, 'bootstrap_features': True, 'max_samples': 0.5, 'n_estimators': 100}


In [65]:
# Accuracy score on test data with hyperparameters
bc = BaggingClassifier({'bootstrap': False, 'bootstrap_features': True, 'max_samples': 0.5, 'n_estimators': 100})
bc.fit(X_train, y_train)

print('Training Accuracy : %.3f'%bc.score(X_train, y_train))
print('Test Accuracy : %.3f'%bc.score(X_test, y_test))

AttributeError: 'dict' object has no attribute 'fit'

*The model is not fitting the dataset as is, we need to choose a different model.*